Url I got the below info from https://www.yelp.com/developers/v3/manage_app?app_created=True

With just API Keys the process to authenticate is:

Get your API Key from the Manage App page.
Put the API Key in the request header as "Authorization: Bearer <YOUR API KEY>".

In [1]:
api_key= 'Y3L0TV0Zd2cMB01-3dTR5ZXnRnvb0-O34qWtelQLvUOdOVvfBQcvhUzWnycGt0aBq_9llOIwOxyv_JKnJMrH0LYvQ2gRquleAeyAvbZCy-EM7DAVzO37xVVDFu-rYXYx'

In [2]:
import requests
import json
import pandas as pd

headers = {'Authorization': 'Bearer %s' % api_key}

def business_search(location, term):
    endpoint_url = "https://api.yelp.com/v3/businesses/search"
    for offset in range(0, 1000, 50):
        params = {
        'limit': 50,
        'term': term.replace(' ', '+'),  #should be restaurants intially
        'location': location.replace(' ', '+'),
        'offset': offset
        }
        if offset == 0:
            results = requests.get(endpoint_url, params = params, headers=headers)
            loaded_json = json.loads(results.text)
            Business_DF = pd.json_normalize(loaded_json['businesses'])
            
        else:
            results = requests.get(endpoint_url, params = params, headers=headers)
            loaded_json = json.loads(results.text)
            add_Business_DF = pd.json_normalize(loaded_json['businesses'])
            Business_DF = Business_DF.append(add_Business_DF)
            
   
    return(Business_DF)

def buiness_reviews(yelp_id):
    endpoint_url = f"https://api.yelp.com/v3/businesses/{yelp_id}/reviews"
    results = requests.get(endpoint_url, headers=headers)
    loaded_json = json.loads(results.text)
    Review_DF = pd.json_normalize(loaded_json['reviews'])
    return(Review_DF)

In [3]:
Missoula_rest = business_search('Missoula', 'restaurants')

In [4]:
len(Missoula_rest)

239

In [5]:
Missoula_rest.columns

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address'],
      dtype='object')

In [11]:
#Missoula_rest[Missoula_rest['is_closed'] != False]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address


In [13]:
Missoula_rest.drop(['alias', 'image_url', 'is_closed', 'display_phone', 'distance', 'categories', 'transactions', 'coordinates.latitude',
                    'coordinates.longitude', 'location.address3', 'location.country', 'location.state', 
                    'location.display_address'], axis=1, inplace=True)

In [14]:
Missoula_rest.rename(columns = {'location.zip_code':'Zip_Code', 
                              'location.city':'City',
                              'location.address2': 'Address_2',
                              'location.address1' : 'Address'}, inplace = True)

In [15]:
Missoula_rest = Missoula_rest.set_index('name')

In [17]:
#Missoula_rest.sort_values('rating',  ascending=False)

In [18]:
not_missoula = Missoula_rest[Missoula_rest['City'] != 'Missoula'].index
Missoula_rest.drop(not_missoula, inplace = True)

In [22]:
no_address1 = Missoula_rest[Missoula_rest['Address'].isna()].index
Missoula_rest.drop(no_address1, inplace = True)

In [23]:
no_address2 = Missoula_rest[Missoula_rest['Address'] == ''].index
Missoula_rest.drop(no_address2, inplace = True)

In [36]:
Missoula_rest.to_csv('Yelp_Missoula_Restaurants.csv')

In [25]:
review_dictionary_yelp = {}
for key, value in Missoula_rest.iterrows():
    review_dictionary_yelp[key] = buiness_reviews(value['id'])

In [30]:
for key in review_dictionary_yelp:
    review_dictionary_yelp[key].drop(['id', 'url', 'user.id', 'user.profile_url', 'user.image_url'],
                                     axis=1, inplace=True)
    review_dictionary_yelp[key].rename(columns = {'user.name':'Name'}, inplace = True)

In [31]:
review_dictionary_yelp['Boxcar Bistro']

,text,rating,time_created,Name
0,We had an amazing dining experience here last ...,5,2021-10-28 16:57:18,Ashley H.
1,Love the ambiance in this restaurant. Nice hig...,5,2021-10-13 18:03:23,Troy M.
2,This is a much needed addition to the neighbor...,5,2021-09-29 19:14:19,Jocy M.


In [35]:
for key in review_dictionary_yelp: 
    review_dictionary_yelp[key].to_csv('Yelp_Reviews/' + str(key) + '_Yelp_reviews.csv')

In [37]:
len(Missoula_rest)

208